In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/anime-gender-classification/dataset/val.csv
/kaggle/input/anime-gender-classification/dataset/train.csv
/kaggle/input/anime-gender-classification/dataset/test.csv
/kaggle/input/anime-gender-classification/dataset/submission_format.csv
/kaggle/input/anime-gender-classification/dataset/val_images/masculine_01931.jpg
/kaggle/input/anime-gender-classification/dataset/val_images/feminine_01838.jpg
/kaggle/input/anime-gender-classification/dataset/val_images/feminine_02355.jpg
/kaggle/input/anime-gender-classification/dataset/val_images/feminine_00265.jpg
/kaggle/input/anime-gender-classification/dataset/val_images/feminine_02670.jpg
/kaggle/input/anime-gender-classification/dataset/val_images/masculine_00313.jpg
/kaggle/input/anime-gender-classification/dataset/val_images/masculine_00617.jpg
/kaggle/input/anime-gender-classification/dataset/val_images/feminine_02864.jpg
/kaggle/input/anime-gender-classification/dataset/val_images/masculine_01478.jpg
/kaggle/input/anime-gender-

In [30]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"device used: {device}")

device used: cpu


###  **Preparing for Datatset**

In [31]:
root_path = '/kaggle/input/anime-gender-classification/dataset'
# Image directories
img_paths = {
    'train': os.path.join(root_path, "train_images"),
    'test': os.path.join(root_path, "test_images"),
    'val': os.path.join(root_path, "val_images")
}

# Label files
label_paths = {
    'train': os.path.join(root_path, "train.csv"),
    'test': os.path.join(root_path, "test.csv"),
    'val': os.path.join(root_path, "val.csv")
}

#check the count of the image directory
def check_count(path):
    counts = sum(1 for f in os.scandir(path) if f.is_file())
    return counts

print("Count for train images : ", check_count(img_paths['train']))
print("Count for test images : ", check_count(img_paths['test']))
print("Count for val images : ", check_count(img_paths['val']))

Count for train images :  4800
Count for test images :  600
Count for val images :  600


In [39]:
class AnimeDataset(Dataset):
    def __init__(self, csv_path, image_dir, transform=None, id_col="id", label_col="label"):
        self.image_dir = image_dir
        self.df = pd.read_csv(csv_path)
        self.transform = transform
        self.id_col = id_col
        self.label_col = label_col

    def __len__(self):
        return len(self.df)

    # mapping for image and label
    def __getitem__(self, idx):
        dataset = self.df.iloc[idx]
        img_path = os.path.join(self.image_dir, dataset[self.id_col])
            
        if os.path.isfile(img_path):
            image = Image.open(img_path).convert("RGB")
            image = self.transform(image)
            label = dataset[self.label_col]
        return image, label


###  Image argumentation and preprocessing

In [49]:
train_transform = transforms.Compose([transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

val_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

# set the image for training, valdataion and testing
train_data = AnimeDataset(label_paths['train'], img_paths['train'], train_transform)
val_data = AnimeDataset(label_paths['val'], img_paths['val'], val_transform)
test_data = AnimeDataset(label_paths['test'], img_paths['test'], val_transform)

train_loader = DataLoader(train_data, batch_size = 32, shuffle=True)
val_loader = DataLoader(val_data, batch_size = 32, shuffle=True)
test_loader = DataLoader(test_data, batch_size = 32, shuffle=True)

###   Model Training and Evaluation

In [ ]:
# Define the model
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.last_channel, 2)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    total = 0
    correct = 0
    # training the model
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        is_has, predicted = torch.max(output, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        
    train_acc = (correct / total)

    # model evaluation
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            loss = criterion(output, labels)
            val_loss += loss.item()
            is_has, predicted = torch.max(output, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)
        val_acc = (val_correct / val_total)
    print(f"Epoch [{epoch+1}/{epochs}] | "
            f"Train Loss: {total_loss:.4f} | Train Acc: {train_acc:.2f} | "
             f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}")

###  Model Testing and Submission

In [ ]:
results = []

# Testing my model and 
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)

        output = model(images)
        is_has, predicted = torch.max(output, 1)

        for img_name, pred in zip(img_names, predicted):
            results.append({"id": img_name, "prediction": int(pred.item())})


submission = pd.DataFrame(results)
submission.to_csv("submission.csv", index=False)

submission.head()